In [1]:
# !pip install -U gensim

In [1]:
# !pip install pycocoevalcap --user
# !pip install pycocotools --user
# !pip install rouge-score --user
# !pip install rouge --user
# !pip install git+https://github.com/Maluuba/nlg-eval.git@master --user

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
# from gensim.utils import tokenize, simple_preprocess, simple_tokenize
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import pickle

[nltk_data] Downloading package punkt to /home/ss4yd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_csv('./prepared_prelim_data_cls256.csv')
df.head()

,pid,tissue_type,notes,svs_path,patch_path,reps_path
0,GTEX-1117F-0726,Heart - Atrial Appendage,"2 pieces, no abnormalities",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
1,GTEX-1117F-1026,Lung,"2 pieces, moderate congestion/moderate to mark...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
2,GTEX-1117F-1126,Liver,"2 pieces, subtotal massive hepatic necrosis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
3,GTEX-1117F-1226,Spleen,"2 pieces, marked congestion/autolysis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
4,GTEX-1117F-1326,Adipose - Visceral (Omentum),"2 pieces, diffuse mesothelial hyperplasia; ~10...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...


In [4]:
df['tokens']=df.notes.apply(word_tokenize)

In [5]:
max_len=max(df.tokens.apply(len))
max_len

29

In [6]:
captions = []
word_freq = Counter()
for index, row in df.iterrows():
    # Update word frequency
    word_freq.update(row['tokens'])
    if len(row['tokens']) <= max_len:
        captions.append(row['tokens'])

In [7]:
min_word_freq=1
words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
word_map = {k: v + 1 for v, k in enumerate(words)}
word_map['<unk>'] = len(word_map)+1
word_map['<start>'] = len(word_map)+1
word_map['<end>'] = len(word_map)+1
word_map['<pad>'] = 0

In [8]:
def add_start_end(ls,max_len):
    ls = ['<start>']+ls+['<end>']+['<pad>']*(max_len-len(ls))
    return ls

df['p_tokens']=df.tokens.apply(lambda x: add_start_end(x,max_len))

In [9]:
df.head()['p_tokens'][0]

['<start>',
 '2',
 'pieces',
 ',',
 'no',
 'abnormalities',
 '<end>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

In [10]:
with open('word_map_cls256.pickle', 'wb') as file:
    pickle.dump(word_map, file)

In [11]:
len(word_map)

155

In [12]:
df['caplens']=df.tokens.apply(lambda x: len(x)+2)
df

,pid,tissue_type,notes,svs_path,patch_path,reps_path,tokens,p_tokens,caplens
0,GTEX-1117F-0726,Heart - Atrial Appendage,"2 pieces, no abnormalities",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, no, abnormalities]","[<start>, 2, pieces, ,, no, abnormalities, <en...",7
1,GTEX-1117F-1026,Lung,"2 pieces, moderate congestion/moderate to mark...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, moderate, congestion/moderate, ...","[<start>, 2, pieces, ,, moderate, congestion/m...",10
2,GTEX-1117F-1126,Liver,"2 pieces, subtotal massive hepatic necrosis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, subtotal, massive, hepatic, nec...","[<start>, 2, pieces, ,, subtotal, massive, hep...",9
3,GTEX-1117F-1226,Spleen,"2 pieces, marked congestion/autolysis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, marked, congestion/autolysis]","[<start>, 2, pieces, ,, marked, congestion/aut...",7
4,GTEX-1117F-1326,Adipose - Visceral (Omentum),"2 pieces, diffuse mesothelial hyperplasia; ~10...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, diffuse, mesothelial, hyperplas...","[<start>, 2, pieces, ,, diffuse, mesothelial, ...",16
...,...,...,...,...,...,...,...,...,...
118,GTEX-11DXX-2626,Artery - Tibial,"2 pieces, focal medial calcification, atherosc...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, focal, medial, calcification, ,...","[<start>, 2, pieces, ,, focal, medial, calcifi...",18
119,GTEX-11DXY-0226,Testis,"2 pieces, decreased spermatogenesis and some S...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, decreased, spermatogenesis, and...","[<start>, 2, pieces, ,, decreased, spermatogen...",12
120,GTEX-11DXY-0726,Pancreas,"2 pieces, fibrosis and atrophy, larger piece i...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, fibrosis, and, atrophy, ,, larg...","[<start>, 2, pieces, ,, fibrosis, and, atrophy...",24
121,GTEX-11DXY-0826,Heart - Left Ventricle,"2 pieces, few box car nuclei (myocyte hypertro...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, few, box, car, nuclei, (, myocy...","[<start>, 2, pieces, ,, few, box, car, nuclei,...",20


In [13]:
msk = np.random.rand(len(df)) < 0.9
# ndf=f.reset_index()
train = df[msk].copy()
temptest = df[~msk].copy()

tst_mask=np.random.rand(len(temptest)) < 0.5
test = temptest[tst_mask].copy()
val = temptest[~tst_mask].copy()

train['dtype']='train'
test['dtype']='test'
val['dtype']='val'

newndf=pd.concat([train,test,val])
newndf.dtype.value_counts()

train    110
val        8
test       5
Name: dtype, dtype: int64

In [14]:
newndf['idx_tokens']=newndf.p_tokens.apply(lambda x: [word_map[i] if i in word_map.keys() else word_map['<unk>'] for i in x])
newndf

,pid,tissue_type,notes,svs_path,patch_path,reps_path,tokens,p_tokens,caplens,dtype,idx_tokens
0,GTEX-1117F-0726,Heart - Atrial Appendage,"2 pieces, no abnormalities",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, no, abnormalities]","[<start>, 2, pieces, ,, no, abnormalities, <en...",7,train,"[153, 1, 2, 3, 4, 5, 154, 0, 0, 0, 0, 0, 0, 0,..."
1,GTEX-1117F-1026,Lung,"2 pieces, moderate congestion/moderate to mark...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, moderate, congestion/moderate, ...","[<start>, 2, pieces, ,, moderate, congestion/m...",10,train,"[153, 1, 2, 3, 6, 152, 7, 8, 9, 154, 0, 0, 0, ..."
2,GTEX-1117F-1126,Liver,"2 pieces, subtotal massive hepatic necrosis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, subtotal, massive, hepatic, nec...","[<start>, 2, pieces, ,, subtotal, massive, hep...",9,train,"[153, 1, 2, 3, 152, 152, 10, 11, 154, 0, 0, 0,..."
3,GTEX-1117F-1226,Spleen,"2 pieces, marked congestion/autolysis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, marked, congestion/autolysis]","[<start>, 2, pieces, ,, marked, congestion/aut...",7,train,"[153, 1, 2, 3, 8, 12, 154, 0, 0, 0, 0, 0, 0, 0..."
4,GTEX-1117F-1326,Adipose - Visceral (Omentum),"2 pieces, diffuse mesothelial hyperplasia; ~10...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, diffuse, mesothelial, hyperplas...","[<start>, 2, pieces, ,, diffuse, mesothelial, ...",16,train,"[153, 1, 2, 3, 13, 14, 15, 16, 17, 18, 152, 19..."
...,...,...,...,...,...,...,...,...,...,...,...
58,GTEX-117XS-0726,Heart - Atrial Appendage,"2 pieces, myocardial infarction, diffuse inte...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, myocardial, infarction, ,, diff...","[<start>, 2, pieces, ,, myocardial, infarction...",11,val,"[153, 1, 2, 3, 128, 129, 3, 13, 24, 73, 154, 0..."
84,GTEX-1192W-0626,Lung,"2 pieces, marked congestion/possible hemorrhag...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, marked, congestion/possible, he...","[<start>, 2, pieces, ,, marked, congestion/pos...",9,val,"[153, 1, 2, 3, 8, 152, 152, 152, 154, 0, 0, 0,..."
91,GTEX-1192W-2126,Prostate,"2 pieces, both with glandular and stromal hype...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, both, with, glandular, and, str...","[<start>, 2, pieces, ,, both, with, glandular,...",16,val,"[153, 1, 2, 3, 98, 48, 89, 47, 90, 15, 16, 30,..."
121,GTEX-11DXY-0826,Heart - Left Ventricle,"2 pieces, few box car nuclei (myocyte hypertro...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, few, box, car, nuclei, (, myocy...","[<start>, 2, pieces, ,, few, box, car, nuclei,...",20,val,"[153, 1, 2, 3, 135, 152, 152, 152, 20, 137, 13..."


In [15]:
newndf.to_pickle('prepared_prelim_data_tokenized_cls256.pickle')